In [1]:
import cudf as pd
from sklearn.datasets import load_boston
boston = load_boston()
df = pd.DataFrame(data=boston.data, columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
        'TAX', 'PTRATIO', 'B', 'LSTAT'])
df['target'] = boston.target
df.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [2]:
import cupy as np
X=df.iloc[:,0:13]
Y=df.iloc[:,13]
Y=np.array(Y)
Y=Y.reshape(-1,1)

In [3]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X.as_matrix())

In [4]:
X_scaled

array([[-0.41978194,  0.28482986, -1.2879095 , ..., -1.45900038,
         0.44105193, -1.0755623 ],
       [-0.41733926, -0.48772236, -0.59338101, ..., -0.30309415,
         0.44105193, -0.49243937],
       [-0.41734159, -0.48772236, -0.59338101, ..., -0.30309415,
         0.39642699, -1.2087274 ],
       ...,
       [-0.41344658, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.98304761],
       [-0.40776407, -0.48772236,  0.11573841, ...,  1.17646583,
         0.4032249 , -0.86530163],
       [-0.41500016, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.66905833]])

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.3, random_state=42)

In [6]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(354, 13)
(152, 13)
(354, 1)
(152, 1)


In [7]:
from cuml import LinearRegression
import cuml

In [8]:
alg = ["eig", "svd", "qr", "svd-qr", "svd-jacobi"]
for i in alg:
    lr = LinearRegression(fit_intercept = True, normalize = False,
                      algorithm = i)
    reg = lr.fit(X_train,y_train)
    preds = lr.predict(X_test)
    print(cuml.metrics.regression.mean_squared_error(y_test,preds))
    print(cuml.metrics.regression.r2_score(y_test,preds))
    print(cuml.metrics.regression.mean_absolute_error(y_test,preds))
    print()

139.59932404504923
0.711226005748492
8.983415627235035

139.59932404504897
0.7112260057484925
8.983415627235024

139.59932404504897
0.7112260057484925
8.983415627235024

77.29709884147577
-0.06387682173517528
6.538584813856789

77.29709884147577
-0.06387682173517528
6.538584813856789

